# TODO: club numbers filter (while scraping)

In [264]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [285]:
def get_club_nums_list():
    ''' Returns a list of all possible club numbers from WorldClass page'''
    
    URL = "https://www.worldclass.ru/trainers"
    page = requests.get(URL)

    soup = BeautifulSoup(page.text, "html.parser")
    num_list = soup.find_all("option", attrs={"data-code": True})
    num_list = [num.attrs['value'] for num in num_list]
    
    return num_list

In [286]:
def get_clubs_strings(all_num_list, ratio=0.5):
    ''' 
    Returns two strings with club numbers divided by a comma.
    
        Parameters:
                    all_num_list (list): A list of all possible club numbers
                    ratio (float): Desired split (from 0 to 1) of numbers from all_num_list (begining from the first element)
                    
        Returns: 
                    clubs_string_first (str): String of numbers before the split
                    clubs_string_second (str): String of numbers after the split
                    
    '''
    
    num_list_first = all_num_list[:int(len(num_list) * ratio)]
    clubs_string_first = ",".join(num_list_first)
    
    num_list_second = all_num_list[int(len(num_list) * ratio):]
    clubs_string_second = ",".join(num_list_second)
    
    return clubs_string_first, clubs_string_second

In [287]:
all_num_list = get_club_nums_list()
clubs_string_first, clubs_string_second = get_clubs_strings(all_num_list)

In [309]:
def get_trainer_list(clubs_string, les_mills=False):
    '''
    Returns a list of trainers from clubs_string clubs.

            Parameters:
                    clubs_string (str): A string with club numbers divided by a comma (e.g. '1,2,3,4,5')
                    les_mills (bool): If True, list will contain only Les Mills programms trainers

            Returns:
                    trainers_list (list): A list of (name, rating, position, link) tuples
                    
    '''
    
    # building URL based on arguments
    URL = "https://www.worldclass.ru/trainers/?club=" + clubs_string
    URL = URL + "&programm[]=93&programm[]=programmy-les-mills" if les_mills else URL
    URL = URL + "&SHOWALL_1=1"
    
    page = requests.get(URL)
    
    soup = BeautifulSoup(page.text, "html.parser")
    
    # retrieving raw html format list of trainers from that page
    trainers_list = soup.find_all("div", class_="trainers__item")
    
    # building tuples (name, rating, position, href_link) from raw list
    trainers_list = [(trainer.find(class_="staff-card__name").text, trainer.find(class_="span staff-card__rating-count"), 
                   trainer.find(class_="staff-card__text").text, trainer.find(class_ ="staff-card__name").attrs['href']) 
                  for trainer in trainers_list]
    
    # removing whitespaces from names, getting ratings (strings for now) if they exist (otherwise, assigning 0s), making links
    trainers_list = [(name.strip(), rating.text, position, "https://www.worldclass.ru" + link) if rating 
                  else (name, 0, position, "https://www.worldclass.ru" + link) 
                  for (name, rating, position, link) in trainers_list]
    
    # ratings to integers, dealing with bad ratings data (instances of "<l" instead of a number or None)
    trainers_list = [(name, int(rating), position, link) if rating != '<l' 
                  else (name, 0, position, link) 
                  for (name, rating, position, link) in trainers_list]
    
    return trainers_list

In [310]:
trainers_list_first = get_trainer_list(clubs_string_first)
trainers_list_second = get_trainer_list(clubs_string_second)

In [311]:
# appending two extracted lists, getting rid of duplicates
full_list = list(set(trainers_list_first + trainers_list_second))

In [312]:
# creating a dataframe with all our information from a list
df = pd.DataFrame(full_list, columns=['Тренер', 'Количество голосов', 'Должность', 'Ссылка'])

In [378]:
df.head()

,Тренер,Количество голосов,Должность,Ссылка,Клубы
0,Александр Жиров,0,Персональный тренер тренажерного зала,https://www.worldclass.ru/trainers/zhirov-alek...,[Триумф]
1,Илья Никитин,0,Персональный тренер тренажерного зала,https://www.worldclass.ru/trainers/ilya-nikitin/,[Триумф]
2,Татьяна Назарова,1,Персональный тренер тренажерного зала,https://www.worldclass.ru/trainers/nazarova-ta...,[Стачек]
3,Анна Хусаинова,1,Супервайзер детского фитнеса. Мастер-тренер де...,https://www.worldclass.ru/trainers/khusainova-...,[Пресня]
4,Сергей Новицкий,4,Персональный тренер водных программ,https://www.worldclass.ru/trainers/novitskiy-s...,[Капитолий]


## Extracting club names

In [333]:
def extract_club_names(URL):
    '''
    Returns a list of clubs from trainer's page (URL).

            Parameters:
                    URL (str): URL of some trainer's page
                    
            Returns:
                    club_list (list): A list of club names
                    
    '''
    page = requests.get(URL)
    soup = BeautifulSoup(page.text, "html.parser")
    
    club_list = soup.find_all(class_="club_link")
    club_list = [club.text for club in club_list]
    
    return club_list

In [335]:
# retrieving club names from the trainer's page --> new column
df['Клубы'] = df['Ссылка'].apply(extract_club_names)

## Some manipulations with data

In [368]:
# sorting
df_sorted_rating = df.sort_values('Количество голосов', ascending=False)

In [372]:
# filtering by club names
club_list = ['Воробьевы горы', 'Капитолий']
df_by_clubs = df_sorted_rating[df_sorted_rating['Клубы'].apply(lambda x: any(club in x for club in club_list))]

In [376]:
df_by_clubs.head(60)

,Тренер,Количество голосов,Должность,Ссылка,Клубы
491,Мария Чернобай,51,"Персональный тренер групповых программ, персон...",https://www.worldclass.ru/trainers/chernobay-m...,[Капитолий]
626,Валерия Кехриниотис,20,Мастер-тренер тренажёрного зала,https://www.worldclass.ru/trainers/valeriya-ke...,[Капитолий]
792,Розалия Попова,19,Мастер-тренер групповых программ,https://www.worldclass.ru/trainers/rozaliya-po...,[Капитолий]
204,Эдуард Кунтудаев,15,"Персональный тренер тренажерного зала, фитнес-...",https://www.worldclass.ru/trainers/kuntudaev-e...,[Капитолий]
1047,Дмитрий Шаталов,15,Мастер-тренер Тренажерного зала,https://www.worldclass.ru/trainers/shatalov-dm...,[Воробьевы горы]
1126,Александр Атаманюк,15,"Персональный тренер групповых программ, фитнес...",https://www.worldclass.ru/trainers/atamanyuk-a...,[Капитолий]
590,Карен Фероян,11,Мастер-тренер Боевых искусств,https://www.worldclass.ru/trainers/feroyan-kar...,[Воробьевы горы]
220,Игорь Янкин,9,"Супервайзер, Мастер-тренер Групповых программ,...",https://www.worldclass.ru/trainers/yankin-igor/,[Воробьевы горы]
747,Ирина Богатова,8,Фитнес-тренер Групповых программ,https://www.worldclass.ru/trainers/bogatova-ir...,[Воробьевы горы]
152,Дмитрий Кок,8,Фитнес-тренер тренажерного зала,https://www.worldclass.ru/trainers/dmitriy-kok/,[Капитолий]


### Clickable links

In [363]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

df_sorted_rating.style.format({'Ссылка': make_clickable})

,Тренер,Количество голосов,Должность,Ссылка,Клубы
547,Равиль Сафин,75,Персональный тренер тренажерного зала,https://www.worldclass.ru/trainers/ravil-safin/,['Метрополис']
897,Юсуфи Ясин,60,Мастер-тренер тренажерного зала,https://www.worldclass.ru/trainers/yasin-yusufi/,['Метрополис']
90,Егор Глазатов,58,Супервайзер тренажерного зала. Мастер-тренер тренажёрного зала,https://www.worldclass.ru/trainers/glazatov-egor-valentinovich/,['Оружейный']
491,Мария Чернобай,51,"Персональный тренер групповых программ, персональный тренер программ Mind&Body",https://www.worldclass.ru/trainers/chernobay-mariya/,['Капитолий']
1507,Анастасия Гадалина,48,Персональный тренер водных программ,https://www.worldclass.ru/trainers/gadalina-anastasiya-alekseevna/,['Щелковский']
951,Карина Соколова,47,Персональный тренер групповых программ,https://www.worldclass.ru/trainers/karina-sokolova/,['Центр']
546,Татьяна Яковлева,45,Персональный тренер тренажерного зала,https://www.worldclass.ru/trainers/tatyana-yakovleva/,['Атлантик Сити']
234,Ольга Соловьева,42,Мастер-тренер тренажерного зала,https://www.worldclass.ru/trainers/olga-berdyugina/,['Митино']
297,Павел Катков,41,Персональный тренер водных программ,https://www.worldclass.ru/trainers/pavel-katkov/,['Варшавка']
1422,Алсу Валькова,41,Персональный тренер групповых программ,https://www.worldclass.ru/trainers/valkova-alsu/,['Стачек']


## Saving to .csv

In [377]:
df.to_csv('world_class_trainers.csv')